In [ ]:
%load_ext autoreload
%autoreload 2

In [3]:
import random
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch

from dataloader.rosen import RosenData
from visualize import get_vae, get_model, set_random, build_estimator
config = {
    'estimator': 'nngp',
    'random_seed': 43,
    'n_dim': 10,
    'data_size': 2000,
    'data_split': [0.2, 0.1, 0.1, 0.6],
    'update_size': 100,
    'al_iterations': 10,
    'verbose': True,
    'use_cache': True,
    'layers': [10, 128, 64, 32, 1],
    'patience': 5,
    'retrain': False,
    'model_path': 'model/data/rosen_visual.ckpt'
}


In [ ]:
epochs = 100_000
patience = 5
# rosen = RosenData(
#     config['n_dim'], config['data_size'], config['data_split'],
#     use_cache=config['use_cache'])
rosen = RosenData(config['n_dim'], 20000, config['data_split'], use_cache=False)
x_pool, y_pool = rosen.dataset('pool')
x_train, y_train = rosen.dataset('train')
x_val, y_val = rosen.dataset('train')

set_random(config['random_seed'])
model = get_model(config['retrain'], config['model_path'], (x_train, y_train), (x_val, y_val))
estimator = build_estimator(config['estimator'], model)
estimation = estimator.estimate(x_pool, x_train, y_train)




In [ ]:
vae = get_vae(restore=False, rosen=rosen, patience=patience)

# Check vae
x_batch = x_train[:30]
decoded = vae.predict(x_batch)
for i in range(5):
    plt.scatter([x[i] for x in x_batch], [y[i] for y in decoded])
plt.show()


In [ ]:
# Make 3d picture of pool points
points = 500
x_batch = x_pool[:points]
vae.eval()
with torch.no_grad():
    surface = vae(torch.DoubleTensor(x_batch))[1].tolist()
xs = [point[0] for point in surface]
ys = [point[1] for point in surface]

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
Axes3D(fig).scatter(xs, ys, zs=y_pool[:points])
plt.show()